In [5]:
print("Hello LangChain")

Hello LangChain


In [6]:
#!pip install langchain_openai

In [7]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI 

#load_dotenv(dotenv_path='.env')
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
#print(OPENAI_API_KEY[:10])
print(OPENAI_API_KEY[:5])

sk-pr


In [8]:
# prompt + llm + output 

# prompt: (쉽게 말해) AI에게 하는 질문
prompt = ChatPromptTemplate.from_messages(
    [ ("system", "당신은 개발자입니다.") , # AI에게 역할을 주는 것.
     ("user", "{input}") ]
)
print(type(prompt))
print(prompt)

<class 'langchain_core.prompts.chat.ChatPromptTemplate'>
input_variables=['input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 개발자입니다.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]


In [9]:
prompt_text = prompt.format(input="파이썬은 무엇인가요? 자세하게 설명해주세요")
print(type(prompt_text))
print(prompt_text)

<class 'str'>
System: 당신은 개발자입니다.
Human: 파이썬은 무엇인가요? 자세하게 설명해주세요


In [10]:
#llm = ChatOpenAI(api_key=OPENAI_API_KEY, model_name="gpt-3.5-turbo-0125")

# Groq는 ChatGPT와 호환 -> ChatGPT 모델 사용 가능
# Groq API를 사용하는 ChatOpenAI 인스턴스 생성 (모델 가져와서 생성)
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct", # 사용 모델: llama-4-scout
    #model="moonshotai/kimi-k2-instruct-0905", # 사용 모델: kimi-k2
    model="openai/gpt-oss-120b", # 사용 모델: gpt-oss-120b
    temperature=0.7
)
print(type(llm)) # 타입: ChatOpenAI
print(llm)

<class 'langchain_openai.chat_models.base.ChatOpenAI'>
client=<openai.resources.chat.completions.completions.Completions object at 0x10f7439a0> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x10f7ec280> root_client=<openai.OpenAI object at 0x10f743e20> root_async_client=<openai.AsyncOpenAI object at 0x118390eb0> model_name='openai/gpt-oss-120b' temperature=0.7 model_kwargs={} openai_api_key=SecretStr('**********') openai_api_base='https://api.groq.com/openai/v1'


In [11]:
# 생성한 모델에 질문하기(prompt 입력)
try:
    response = llm.invoke(prompt_text)
    print(type(response))
    print("응답:", response.content)
except Exception as e:
    print(f"오류 발생: {e}")

오류 발생: Error code: 401 - {'error': {'message': 'Invalid API Key', 'type': 'invalid_request_error', 'code': 'invalid_api_key'}}


# LCEL

In [12]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [13]:
chain = prompt | llm | output_parser
print(type(chain))
print(chain)

<class 'langchain_core.runnables.base.RunnableSequence'>
first=ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 개발자입니다.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]) middle=[ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x10f7439a0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x10f7ec280>, root_client=<openai.OpenAI object at 0x10f743e20>, root_async_client=<openai.AsyncOpenAI object at 0x118390eb0>, model_name='openai/gpt-oss-120b', temperature=0.7, model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://api.groq.com/openai/v1')] last=StrOutputParser()


In [ ]:
response = chain.invoke({"input":"LangChain이 무엇인가요?"})

In [ ]:
print(type(response))
print(response)

<class 'str'>
## LangChain이란?

**LangChain**은 **대규모 언어 모델(LLM)**을 활용한 **애플리케이션**을 보다 쉽게 설계·구현·배포할 수 있도록 도와주는 **프레임워크**이자 **툴킷**입니다. 주로 파이썬(Python)과 자바스크립트(TypeScript/Node.js) 환경에서 사용되며, LLM을 단순히 “텍스트를 생성”하는 수준을 넘어 **복합적인 워크플로우**와 **외부 시스템 연동**까지 포괄하는 애플리케이션을 만들 때 유용합니다.

---

## 핵심 개념 & 구성 요소

| 구성 요소 | 설명 | 주요 역할 |
|-----------|------|-----------|
| **Chains** | 여러 LLM 호출·프롬프트·툴을 순차·조건부로 연결한 파이프라인 | 복잡한 로직을 단계별로 나누어 재사용 가능 |
| **Agents** | 목표(Goal)와 툴(tool) 집합을 받아, LLM이 “어떤 툴을 언제 사용할지” 스스로 판단하도록 하는 구조 | 동적 의사결정·툴 호출(검색, DB, API 등) |
| **Prompt Templates** | 프롬프트를 변수화·재사용 가능하게 만든 템플릿 | 일관된 프롬프트 관리·다양한 입력에 맞춤 |
| **Memory** | 대화·세션의 상태를 저장·조회하는 메커니즘 | 컨텍스트 유지·연속적인 대화 흐름 |
| **Indexes / Vector Stores** | 텍스트를 임베딩해 벡터 DB에 저장하고 검색하는 기능 | RAG(Retrieval‑Augmented Generation) 구현 |
| **Callbacks** | 체인·에이전트 실행 시점에 로깅·모니터링·디버깅을 위한 훅 | 트레이스·성능 분석·에러 처리 |
| **Integrations** | OpenAI, Anthropic, Cohere, HuggingFace, Azure 등 다양한 LLM 제공자와의 연결 | 모델 교체·멀티‑모델 전략 |
| **Utilities** | 텍스트 파싱, JSON 변환, 데이터